<a href="https://colab.research.google.com/github/ArushiG11/Anime-Data-Analysis/blob/main/Anime_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

My Anime List : https://myanimelist.net/topanime.php

In [60]:
limit = 0

while(limit<500):
  urls_mal = f'https://myanimelist.net/topanime.php?limit={limit}'
  page = requests.get(urls_mal)
  soup = BeautifulSoup(page.text,'html')

  table = soup.find('table', class_ = 'top-ranking-table')

  if limit == 0:
    table_titles = table.find_all('tr')[0]
    table_titles = table_titles.find_all('td')
    table_titles= [title.text.strip() for title in table_titles]

    df = pd.DataFrame(columns = table_titles)

  table_data = table.find_all('tr')[1:]

  for row in table_data:
    row_data = row.find_all('td')
    row_data = [data.text.strip() for data in row_data]
    length = len(df)
    df.loc[length] = row_data

  limit += 50

df = df[["Rank", "Title", "Score"]]
df.to_csv('myanimelist.csv', index= False)

aniDB : https://anidb.net/anime/?h=1&noalias=1&orderby.name=1.1&orderby.rating=0.2&page=1&view=list

In [66]:
urls_anidb = ['https://anidb.net/anime/?h=1&noalias=1&orderby.name=1.1&orderby.rating=0.2&view=list']
for p in range(1,410):
    urls_anidb.append("https://anidb.net/anime/?h=1&noalias=1&orderby.name=0.1&page="+str(p)+"&view=list")

# behave like browser to avoid restrictions
# mimic te browser request
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}



In [67]:
dfs_anidb= []

In [69]:
r_anidb = requests.get(urls_anidb[0], headers=header)

In [71]:
soup = BeautifulSoup(r_anidb.text,'html')

In [72]:
soup

<!DOCTYPE html>
<html lang="en" prefix="og: http://ogp.me/ns#">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>AniDB AntiLeech - AniDB</title>
<meta content="Looking for information about AniDB AntiLeech? AniDB is the right place for you. AniDB is a not-for-profit anime database providing you with all infor..." name="description" property="og:description"/>
<meta content="index, follow" name="robots"/>
<link href="https://cdn-us.anidb.net/css/anidbstyle3/anidbstyle3.css?v=2023-11-02T17-27-31" rel="stylesheet" title="Style" type="text/css"/>
<link href="https://cdn-us.anidb.net/css/assets/images/touch/apple-touch-icon.png?v=2023-11-02T17-27-31" rel="apple-touch-icon" sizes="180x180"/>
<link href="https://cdn-us.anidb.net/css/assets/images/touch/favicon-32x32.png?v=2023-11-02T17-27-31" rel="icon" sizes="32x32" type="image/png"/>
<link href="https://cdn-us.anidb.net/css/assets/images/

In [ ]:
for idx,i in enumerate(urls_anidb):
    r_anidb = requests.get(i, headers=header)
    dfs_anidb.append(pd.read_html(r_anidb.text))
    print("Anidb page is scanned: " + str(idx+1) + "/" + str(len(urls_anidb)), end='\r')

In [75]:
import requests
import pandas as pd

# Hypothetical URL of an internal API endpoint discovered through network inspection
api_url_template = "http://api.anidb.net:9001/httpapi?client={str}&clientver={int}&protover=1"

# Headers to mimic a browser request
header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

# Initialize list for DataFrames
dfs_anidb = []

# Fetch data from the internal API
for page in range(1, 411):
    api_url = api_url_template.format(page=page)
    response = requests.get(api_url, headers=header)
    data = response.json()  # Assuming the API returns JSON data

    # Convert the JSON data to a DataFrame and append to the list
    df = pd.DataFrame(data['results'])  # Adjust based on actual JSON structure
    dfs_anidb.append(df)

    print(f"AniDB API page {page}/410 scanned", end='\r')

print("")

# Combine all DataFrames into a single DataFrame
all_data_anidb = pd.concat(dfs_anidb, ignore_index=True)

# Now all_data_anidb contains the combined data from all pages


KeyError: 'str'